In [84]:
import pandas as pd
import seaborn as sns
import numpy as np
from statistics import mean
import matplotlib.pyplot as plt
import warnings
from sklearn.preprocessing import PowerTransformer
import numpy as np
from sklearn.preprocessing import LabelEncoder,OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.utils import resample

from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier, GradientBoostingClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.metrics import accuracy_score, classification_report,ConfusionMatrixDisplay, \
                            precision_score, recall_score, f1_score, roc_auc_score,roc_curve,confusion_matrix


from sklearn import metrics 
from sklearn.model_selection import  train_test_split, RepeatedStratifiedKFold, cross_val_score
from sklearn.preprocessing import OneHotEncoder, MinMaxScaler
from sklearn.compose import ColumnTransformer
from sklearn.impute import SimpleImputer, KNNImputer
from xgboost import XGBClassifier
from sklearn.preprocessing import StandardScaler, MinMaxScaler,RobustScaler
from sklearn.compose import ColumnTransformer
from catboost import CatBoostClassifier

warnings.filterwarnings("ignore")
%matplotlib inline

In [83]:
pip install catboost

     -------------------------------------- 101.0/101.0 MB 7.3 MB/s eta 0:00:00
     ---------------------------------------- 47.0/47.0 kB ? eta 0:00:00
Note: you may need to restart the kernel to use updated packages.


In [2]:
df=pd.read_csv('adult.csv')

In [3]:
df.head()

,age,workclass,fnlwgt,education,education-num,marital-status,occupation,relationship,race,sex,capital-gain,capital-loss,hours-per-week,country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [4]:
df.duplicated().sum()

24

In [5]:
df.drop_duplicates(inplace=True)

In [6]:
df.rename(columns={'education-num':"education_num",
                   "marital-status":"marital_status",
                   "capital-gain":"capital_gain",
                   "capital-loss":"capital_loss",
                   "hours-per-week":"hours_per_week"
                   },inplace=True)

In [7]:
df.head()

,age,workclass,fnlwgt,education,education_num,marital_status,occupation,relationship,race,sex,capital_gain,capital_loss,hours_per_week,country,salary
0,39,State-gov,77516,Bachelors,13,Never-married,Adm-clerical,Not-in-family,White,Male,2174,0,40,United-States,<=50K
1,50,Self-emp-not-inc,83311,Bachelors,13,Married-civ-spouse,Exec-managerial,Husband,White,Male,0,0,13,United-States,<=50K
2,38,Private,215646,HS-grad,9,Divorced,Handlers-cleaners,Not-in-family,White,Male,0,0,40,United-States,<=50K
3,53,Private,234721,11th,7,Married-civ-spouse,Handlers-cleaners,Husband,Black,Male,0,0,40,United-States,<=50K
4,28,Private,338409,Bachelors,13,Married-civ-spouse,Prof-specialty,Wife,Black,Female,0,0,40,Cuba,<=50K


In [8]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 32537 entries, 0 to 32560
Data columns (total 15 columns):
 #   Column          Non-Null Count  Dtype 
---  ------          --------------  ----- 
 0   age             32537 non-null  int64 
 1   workclass       32537 non-null  object
 2   fnlwgt          32537 non-null  int64 
 3   education       32537 non-null  object
 4   education_num   32537 non-null  int64 
 5   marital_status  32537 non-null  object
 6   occupation      32537 non-null  object
 7   relationship    32537 non-null  object
 8   race            32537 non-null  object
 9   sex             32537 non-null  object
 10  capital_gain    32537 non-null  int64 
 11  capital_loss    32537 non-null  int64 
 12  hours_per_week  32537 non-null  int64 
 13  country         32537 non-null  object
 14  salary          32537 non-null  object
dtypes: int64(6), object(9)
memory usage: 4.0+ MB


In [9]:
# education and education-num column describe the same so we should drop any one of them
# othser this are having negative correlation so we deleting 
df.drop(labels=['education','fnlwgt','marital_status','relationship'],axis=1,inplace=True)

In [11]:
def get_change(test):
    for req in test:
        df[req].replace(" ?",np.NaN,inplace=True)
        
        
get_change(df)

In [12]:
df.isna().sum()

age                  0
workclass         1836
education_num        0
occupation        1843
race                 0
sex                  0
capital_gain         0
capital_loss         0
hours_per_week       0
country            582
salary               0
dtype: int64

In [24]:
df[df.columns[df.isnull().any()]]

,workclass,occupation,country
0,State-gov,Adm-clerical,United-States
1,Self-emp-not-inc,Exec-managerial,United-States
2,Private,Handlers-cleaners,United-States
3,Private,Handlers-cleaners,United-States
4,Private,Prof-specialty,Cuba
...,...,...,...
32556,Private,Tech-support,United-States
32557,Private,Machine-op-inspct,United-States
32558,Private,Adm-clerical,United-States
32559,Private,Adm-clerical,United-States


In [23]:

df[df['workclass'].isnull()]

,age,workclass,education_num,occupation,race,sex,capital_gain,capital_loss,hours_per_week,country,salary
27,54,NaN,10,NaN,Asian-Pac-Islander,Male,0,0,60,South,>50K
61,32,NaN,4,NaN,White,Male,0,0,40,NaN,<=50K
69,25,NaN,10,NaN,White,Male,0,0,40,United-States,<=50K
77,67,NaN,6,NaN,White,Male,0,0,2,United-States,<=50K
106,17,NaN,6,NaN,White,Female,34095,0,32,United-States,<=50K
...,...,...,...,...,...,...,...,...,...,...,...
32530,35,NaN,13,NaN,White,Female,0,0,55,United-States,>50K
32531,30,NaN,13,NaN,Asian-Pac-Islander,Female,0,0,99,United-States,<=50K
32539,71,NaN,16,NaN,White,Male,0,0,10,United-States,>50K
32541,41,NaN,9,NaN,Black,Female,0,0,32,United-States,<=50K


In [25]:
# dropping null values in columns  workclass and occupation same axis have null values
df.dropna(subset=['workclass','occupation'],inplace=True)

In [27]:
df['country'].fillna(df['country'].mode(),inplace=True)

In [28]:
df['country']=df['country'].replace({"United-States":" United-States",
                                     "0     United-States\nName: country, dtype: object":" United-States",
                                     " Outlying-US(Guam-USVI-etc)":" United-States"})

In [29]:
df['country'].unique()

array([' United-States', ' Cuba', ' Jamaica', ' India', nan, ' Mexico',
       ' Puerto-Rico', ' Honduras', ' England', ' Canada', ' Germany',
       ' Iran', ' Philippines', ' Poland', ' Columbia', ' Cambodia',
       ' Thailand', ' Ecuador', ' Laos', ' Taiwan', ' Haiti', ' Portugal',
       ' Dominican-Republic', ' El-Salvador', ' France', ' Guatemala',
       ' Italy', ' China', ' South', ' Japan', ' Yugoslavia', ' Peru',
       ' Scotland', ' Trinadad&Tobago', ' Greece', ' Nicaragua',
       ' Vietnam', ' Hong', ' Ireland', ' Hungary', ' Holand-Netherlands'],
      dtype=object)

In [30]:
cate_col=df.select_dtypes(include='O').columns
num_col=df.select_dtypes(exclude='O').columns

In [31]:
def get_split(data):
    for i in data:
        #print(i)
        df[i]=df[i].str.replace(" ","")
        

get_split(cate_col)

In [33]:
df['workclass'].unique()

array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
       'Local-gov', 'Self-emp-inc', 'Without-pay'], dtype=object)

In [ ]:
y=

In [162]:
X = df.drop(labels='salary',axis=1)
y = df['salary'].replace({'<=50K':0, '>50K':1})

In [163]:
X

,age,workclass,education_num,occupation,race,sex,capital_gain,capital_loss,hours_per_week,country
0,39,State-gov,13,Adm-clerical,White,Male,2174,0,40,United-States
1,50,Self-emp-not-inc,13,Exec-managerial,White,Male,0,0,13,United-States
2,38,Private,9,Handlers-cleaners,White,Male,0,0,40,United-States
3,53,Private,7,Handlers-cleaners,Black,Male,0,0,40,United-States
4,28,Private,13,Prof-specialty,Black,Female,0,0,40,Cuba
...,...,...,...,...,...,...,...,...,...,...
32556,27,Private,12,Tech-support,White,Female,0,0,38,United-States
32557,40,Private,9,Machine-op-inspct,White,Male,0,0,40,United-States
32558,58,Private,9,Adm-clerical,White,Female,0,0,40,United-States
32559,22,Private,9,Adm-clerical,White,Male,0,0,20,United-States


In [165]:
y.unique()

array([0, 1], dtype=int64)

In [161]:
y.unique()

array(['<=50K', '>50K'], dtype=object)

In [166]:
cate_col=X.select_dtypes(include='O').columns
num_col=X.select_dtypes(exclude='O').columns

In [171]:
num_col

Index(['age', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week'],
      dtype='object')

In [167]:
cate_col

Index(['workclass', 'occupation', 'race', 'sex', 'country'], dtype='object')

In [168]:
report = {}

def unique_list(data):
    for i in data:
        a=df[i].unique()
        
        report[i]=a
    return report

In [169]:
unique_list(cate_col)

{'workclass': array(['State-gov', 'Self-emp-not-inc', 'Private', 'Federal-gov',
        'Local-gov', 'Self-emp-inc', 'Without-pay'], dtype=object),
 'occupation': array(['Adm-clerical', 'Exec-managerial', 'Handlers-cleaners',
        'Prof-specialty', 'Other-service', 'Sales', 'Craft-repair',
        'Transport-moving', 'Farming-fishing', 'Machine-op-inspct',
        'Tech-support', 'Protective-serv', 'Armed-Forces',
        'Priv-house-serv'], dtype=object),
 'race': array(['White', 'Black', 'Asian-Pac-Islander', 'Amer-Indian-Eskimo',
        'Other'], dtype=object),
 'sex': array(['Male', 'Female'], dtype=object),
 'country': array(['United-States', 'Cuba', 'Jamaica', 'India', nan, 'Mexico',
        'Puerto-Rico', 'Honduras', 'England', 'Canada', 'Germany', 'Iran',
        'Philippines', 'Poland', 'Columbia', 'Cambodia', 'Thailand',
        'Ecuador', 'Laos', 'Taiwan', 'Haiti', 'Portugal',
        'Dominican-Republic', 'El-Salvador', 'France', 'Guatemala',
        'Italy', 'China', '

In [170]:
# here we create pipeline for numerical column and catogorical column for preprocess and missing value and ordinal ranking 

num_pipeline = Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy='median')),
        ("scaler",StandardScaler())
    ]
)


cate_pipeline = Pipeline(
    steps=[
        ("imputer",SimpleImputer(strategy='most_frequent')),
        ("ordinal_encoder",OrdinalEncoder(categories=[report['workclass'],report['occupation'],report['race'],report['sex'],report['country']])),
         ("scaler",StandardScaler())
    ]
)

In [172]:
preprocessor = ColumnTransformer([
    ("num_pipeline",num_pipeline,num_col),
    ("cate_pipeline",cate_pipeline,cate_col)
])

In [173]:
preprocessor

ColumnTransformer(transformers=[('num_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='median')),
                                                 ('scaler', StandardScaler())]),
                                 Index(['age', 'education_num', 'capital_gain', 'capital_loss',
       'hours_per_week'],
      dtype='object')),
                                ('cate_pipeline',
                                 Pipeline(steps=[('imputer',
                                                  SimpleImputer(strategy='most_frequent')),
                                                 ('ordinal_encoder',
                                                  OrdinalEncode...
       'Ecuador', 'Laos', 'Taiwan', 'Haiti', 'Portugal',
       'Dominican-Republic', 'El-Salvador', 'France', 'Guatemala',
       'Italy', 'China', 'South', 'Japan', 'Yugoslavia', 'Peru',
       'Scotland', 'Trinadad&Tobago', 'Greece', 'Nicaragua', 'Vietnam',
       'Hong', 'Ireland', 'Hungary', 'Holand-Netherlands'], dtype=object)])),
                                                 ('scaler', StandardScaler())]),
                                 Index(['workclass', 'occupation', 'race', 'sex', 'country'], dtype='object'))])

In [45]:
num_pipeline

Pipeline(steps=[('imputer', SimpleImputer(strategy='median')),
                ('scaler', StandardScaler())])

In [174]:
X_train,X_test,y_train,y_test=train_test_split(X,
                                               y,
                                               test_size=0.3,
                                               random_state=42)

In [175]:
y_test.unique()

array([0, 1], dtype=int64)

In [176]:
X_train

,age,workclass,education_num,occupation,race,sex,capital_gain,capital_loss,hours_per_week,country
3905,56,Private,4,Handlers-cleaners,White,Male,0,2002,40,United-States
19209,21,Private,10,Prof-specialty,White,Female,1055,0,32,United-States
6755,24,Private,10,Machine-op-inspct,White,Male,0,0,40,United-States
15594,34,Private,14,Tech-support,Black,Female,0,0,40,NaN
19115,45,Private,10,Other-service,White,Male,0,0,40,United-States
...,...,...,...,...,...,...,...,...,...,...
31600,25,State-gov,13,Protective-serv,White,Male,0,0,40,United-States
5764,54,Private,16,Prof-specialty,White,Male,0,1977,50,United-States
918,81,Self-emp-not-inc,9,Exec-managerial,White,Male,0,0,30,United-States
16745,33,Private,9,Handlers-cleaners,White,Male,0,0,20,United-States


In [177]:
X_train.shape

(21485, 10)

In [182]:
X_test.shape

(9209, 10)

In [179]:
y_train.shape

(21485,)

In [183]:
y_test.shape

(9209,)

In [184]:
X_train=preprocessor.fit_transform(X_train)
X_test=preprocessor.transform(X_test)

In [136]:

scores = []
def model_evaluation(X_train,X_test,y_train,y_test,models):
    model_rep = {}
    training_rep = {}
    
    for i in range(len(models)):
        model=list(models.values())[i]
        model.fit(X_train,y_train)
        
        y_pred = model.predict(X_test)
        
        accu=accuracy_score(y_test,y_pred)
        tran=model.score(X_train,y_train)
        
        model_rep[list(models.keys())[i]]=accu
        training_rep[list(models.keys())[i]]=tran
        
        scores.append({
            "modeles":list(models.keys())[i],
            "acc_score":accuracy_score(y_test,y_pred),
            "train_score":model.score(X_train,y_train)
        })
        
    return pd.DataFrame(scores,columns=['modeles','acc_score','train_score'])

In [137]:
model_evaluation(X_train,X_test,y_train,y_test,models)

,modeles,acc_score,train_score
0,LogisticRegression,0.819416,0.820526
1,DecisionTreeClassifier,0.789119,0.966674
2,svc,0.827343,0.838585
3,RandomForestClassifier,0.821913,0.966674
4,knn,0.808014,0.863765


In [ ]:
evaluate_models(X_train,X_test,y_train,y_test,models1)

In [144]:
def evaluate_clf(true, predicted):
    '''
    This function takes in true values and predicted values
    Returns: Accuracy, F1-Score, Precision, Recall, Roc-auc Score
    '''
    acc = accuracy_score(true, predicted) # Calculate Accuracy
    f1 = f1_score(true, predicted) # Calculate F1-score
    precision = precision_score(true, predicted) # Calculate Precision
    recall = recall_score(true, predicted)  # Calculate Recall
    roc_auc = roc_auc_score(true, predicted) #Calculate Roc
    return acc, f1 , precision, recall, roc_auc

In [189]:
# Create a function which can evaluate models and return a report 
def evaluate_models(X_train,X_test,y_train,y_test,models):
    '''
    This function takes in X and y and models dictionary as input
    It splits the data into Train Test split
    Iterates through the given model dictionary and evaluates the metrics
    Returns: Dataframe which contains report of all models metrics with cost
    '''
    # separate dataset into train and test
    #X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2,random_state=42)
    
    
    
    scores=[]
    for i in range(len(list(models))):
        model = list(models.values())[i]
        model.fit(X_train, y_train) # Train model

        # Make predictions
        y_train_pred = model.predict(X_train)
        y_test_pred = model.predict(X_test)

        # Training set performance
        model_train_accuracy, model_train_f1,model_train_precision,\
        model_train_recall,model_train_rocauc_score=evaluate_clf(y_train ,y_train_pred)
        #train_cost = total_cost(y_train, y_train_pred)


        # Test set performance
        model_test_accuracy,model_test_f1,model_test_precision,\
        model_test_recall,model_test_rocauc_score=evaluate_clf(y_test, y_test_pred)
       # test_cost = total_cost(y_test, y_test_pred)

        print(list(models.keys())[i])
        #models_list.append(list(models.keys())[i])
        scores.append({'model name':list(models.keys())[i],
                        "model_train_accuracy":model_train_accuracy,
                        "model_train_f1":model_train_f1,
                        "model_train_rocauc_score":model_train_rocauc_score,
                        "model_test_accuracy":model_test_accuracy,
                        "model_test_f1":model_test_f1,
                        "model_test_precision":model_test_precision,
                        "model_test_recall":model_test_recall,
                        "model_test_rocauc_score":model_test_rocauc_score
                        })
        print('Model performance for Training set')
        print("- Accuracy: {:.4f}".format(model_train_accuracy))
        #training_accuracy_list.append(model_train_accuracy)
        print('- F1 score: {:.4f}'.format(model_train_f1)) 
        print('- Precision: {:.4f}'.format(model_train_precision))
        print('- Recall: {:.4f}'.format(model_train_recall))
        print('- Roc Auc Score: {:.4f}'.format(model_train_rocauc_score))
       # print(f'- COST: {train_cost}.')

        print('----------------------------------')

        print('Model performance for Test set')
        print('- Accuracy: {:.4f}'.format(model_test_accuracy))
        print('- F1 score: {:.4f}'.format(model_test_f1))
        print('- Precision: {:.4f}'.format(model_test_precision))
        print('- Recall: {:.4f}'.format(model_test_recall))
        print('- Roc Auc Score: {:.4f}'.format(model_test_rocauc_score))
        #print(f'- COST: {test_cost}.')
        #cost_list.append(test_cost)
        print('='*35)
        print('\n')
        
    report=pd.DataFrame(scores,columns=['model name',
                        "model_test_accuracy",
                        "model_test_f1",
                        "model_test_precision",
                        "model_test_recall",
                        "model_test_rocauc_score"])
        
    return report

In [190]:
report=evaluate_models(X_train,X_test,y_train,y_test,models1)

Random Forest
Model performance for Training set
- Accuracy: 0.9666
- F1 score: 0.9310
- Precision: 0.9596
- Recall: 0.9042
- Roc Auc Score: 0.9458
----------------------------------
Model performance for Test set
- Accuracy: 0.8220
- F1 score: 0.6113
- Precision: 0.6707
- Recall: 0.5617
- Roc Auc Score: 0.7351


Decision Tree
Model performance for Training set
- Accuracy: 0.9667
- F1 score: 0.9294
- Precision: 0.9837
- Recall: 0.8808
- Roc Auc Score: 0.9380
----------------------------------
Model performance for Test set
- Accuracy: 0.7880
- F1 score: 0.5664
- Precision: 0.5777
- Recall: 0.5556
- Roc Auc Score: 0.7104


Gradient Boosting
Model performance for Training set
- Accuracy: 0.8542
- F1 score: 0.6461
- Precision: 0.8171
- Recall: 0.5343
- Roc Auc Score: 0.7473
----------------------------------
Model performance for Test set
- Accuracy: 0.8452
- F1 score: 0.6273
- Precision: 0.7838
- Recall: 0.5229
- Roc Auc Score: 0.7375


Logistic Regression
Model performance for Training 

In [196]:
report.sort_values(by='model_test_accuracy',ascending=False).T

,6,5,2,7,8,0,3,4,1
model name,CatBoosting Classifier,XGBClassifier,Gradient Boosting,AdaBoost Classifier,Support vector machine,Random Forest,Logistic Regression,K-Neighbors Classifier,Decision Tree
model_test_accuracy,0.851884,0.850581,0.845151,0.840699,0.827343,0.822022,0.819416,0.808014,0.788033
model_test_f1,0.654858,0.661084,0.627287,0.620832,0.567465,0.611335,0.555229,0.579248,0.566415
model_test_precision,0.780929,0.76034,0.783801,0.763024,0.75525,0.670656,0.718837,0.638175,0.577707
model_test_recall,0.563834,0.584749,0.522876,0.523312,0.454466,0.561656,0.452288,0.530283,0.555556
model_test_rocauc_score,0.755666,0.761785,0.737501,0.734682,0.70279,0.735051,0.696783,0.715243,0.710378


In [140]:
models1 = {
    "Random Forest": RandomForestClassifier(),
    "Decision Tree": DecisionTreeClassifier(),
    "Gradient Boosting": GradientBoostingClassifier(),
    "Logistic Regression": LogisticRegression(),
     "K-Neighbors Classifier": KNeighborsClassifier(),
    "XGBClassifier": XGBClassifier(), 
     "CatBoosting Classifier": CatBoostClassifier(verbose=False),
    "AdaBoost Classifier": AdaBoostClassifier(),
    "Support vector machine":SVC()
}

## Observation
1.Cat boost classifier is best model
CatBoost is a machine learning library that specializes in handling categorical features efficiently. It provides an implementation of gradient-boosted decision trees designed for high performance and efficiency. The CatBoostClassifier is specifically tailored for classification tasks.

In [ ]:
from sklearn.model_selection import GridSearchCV,RandomizedSearchCV
model = CatBoostClassifier()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'depth': [4, 6, 8],
    'learning_rate': [0.01, 0.05, 0.1],
    'iterations': [100, 200, 300]
}

# Create a GridSearchCV object
Rsv=RandomizedSearchCV(model, param_grid, cv=3, scoring='accuracy')

# Fit the GridSearchCV object to the data
Rsv.fit(X_train, y_train)

# Get the best parameters and best model
best_params = Rsv.best_params_
best_model = Rsv.best_estimator_

# Make predictions using the best model
y_pred = best_model.predict(X_test)

# Evaluate the accuracy
accuracy = accuracy_score(y_test, y_pred)
print(f"Best Parameters: {best_params}")
print(f"Accuracy with Best Model: {accuracy}")

Best Parameters: {'learning_rate': 0.1, 'iterations': 200, 'depth': 4}
Accuracy with Best Model: 0.8514496688022587

In [200]:
model = XGBClassifier()

# Define the parameter grid for hyperparameter tuning
param_grid = {
    'learning_rate': [0.01, 0.1, 0.2],
    'n_estimators': [50, 100, 200],
    'max_depth': [3, 4, 5],
    'subsample': [0.8, 1.0],
    'colsample_bytree': [0.8, 1.0],
    'min_child_weight': [1, 3, 5]
}

# Create a GridSearchCV object
grid_search = GridSearchCV(model, param_grid, cv=3, scoring='accuracy')

# Fit the GridSearchCV object to the data
grid_search.fit(X_train, y_train)

# Get the best parameters and best model
best_params = grid_search.best_params_
best_model = grid_search.best_estimator_
y_pred=best_model.predict(X_test)
# Print the best parameters
print("Best Parameters:", best_params)
print("Best score",grid_search.best_score_)
print("Accuracy",accuracy_score(y_pred,y_test))
print("best estimator",best_model)

Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}
Best score 0.8538981934411215
Accuracy 0.8532956889998914
best estimator XGBClassifier(base_score=None, booster=None, callbacks=None,
              colsample_bylevel=None, colsample_bynode=None,
              colsample_bytree=0.8, device=None, early_stopping_rounds=None,
              enable_categorical=False, eval_metric=None, feature_types=None,
              gamma=None, grow_policy=None, importance_type=None,
              interaction_constraints=None, learning_rate=0.2, max_bin=None,
              max_cat_threshold=None, max_cat_to_onehot=None,
              max_delta_step=None, max_depth=5, max_leaves=None,
              min_child_weight=1, missing=nan, monotone_constraints=None,
              multi_strategy=None, n_estimators=100, n_jobs=None,
              num_parallel_tree=None, random_state=None, ...)


## Best model is XGBClassifier
Best Parameters: {'colsample_bytree': 0.8, 'learning_rate': 0.2, 'max_depth': 5, 'min_child_weight': 1, 'n_estimators': 100, 'subsample': 1.0}
Best score 0.8538981934411215
Accuracy 0.8532956889998914